## General Description

This script reads data from a VTK EnSight Gold binary file and processes it to extract information about solar shading coefficients and the geometry of cells over a series of timesteps. The key steps include:

1. **Setting up the Reader**: The script initializes a VTK reader for the EnSight Gold binary file format and configures it to read all variables from a specified case file.

2. **Processing Data for Each Timestep**: For each timestep, the script updates the reader with the current time value, retrieves cell data including solar shading coefficients, and extracts the geometric points defining each cell. This data is combined and  converted into a pandas DataFrame with appropriate column names.

3. **Calculating the Area of Triangles**: A function calculates the area of a triangle given its vertices. This function is used to compute the area for each triangle (cell) in the DataFrame, and these areas are added as a new column in the DataFrame.

4. **Saving the Data**: The resulting DataFrame is saved to a CSV file for further analysis or use.

This script provides a comprehensive framework for reading, processing, and analyzing time-dependent geometric and scalar data from a VTK EnSight Gold binary file.


In [ ]:
import vtk

# Setup the reader
# We suppose that the necessary files are in the same directory as the notebook
case_file = "strasbourg_sm_lod1/exports/City_Energy_Modeling.case"
# Setup the reader
reader = vtk.vtkEnSightGoldBinaryReader()
reader.SetCaseFileName(case_file)
reader.ReadAllVariablesOn()  # Ensure all variables are read
reader.Update()

# Retrieve the output
output = reader.GetOutput()
timeset=reader.GetTimeSets()
time=timeset.GetItem(0)
timesteps=time.GetSize()

In [ ]:
print("Number of timesteps: ", timesteps)
print("Number of cells: ", reader.GetOutput().GetBlock(0).GetNumberOfCells())
print("Number of scalar points?: ", reader.GetOutput().GetBlock(0).GetCellData().GetArray("solar_shading_coeff").GetNumberOfTuples())

In [ ]:
import numpy as np

In [ ]:
id_reader = vtk.vtkEnSightGoldBinaryReader()
id_reader.SetCaseFileName("strasbourg_sm_lod1/export_ids/City_Energy_Modeling.case")
id_reader.ReadAllVariablesOn()
id_reader.Update()

id_reader.SetTimeValue(id_reader.GetTimeSets().GetItem(0).GetValue(0))
id_reader.Update()
output = id_reader.GetOutput().GetBlock(0)
cell_data = output.GetCellData()
building_ids = cell_data.GetArray("building_id")
building_element_ids = cell_data.GetArray("building_element_id")

bids = []
beids = []

for i in range(output.GetNumberOfCells()):
    bid_part1, bid_part2, bid_part3 = building_ids.GetTuple(i)
    bid_part1 = np.int64(bid_part1)
    bid_part2 = np.int64(bid_part2) << 22
    bid_part3 = np.int64(bid_part3) << 44
    bids.append(bid_part1+bid_part2+bid_part3)

    beids.append(int(building_element_ids.GetTuple(i)[0]))

In [ ]:
print(f"# Building IDs : {np.unique(bids).shape[0]}")
print(f"# Unique building face ids : {np.unique(beids).shape[0]} -- from  {min(beids)} to {max(beids)}")

Get the mesh

In [ ]:
import numpy as np

output = reader.GetOutput().GetBlock(0)
mesh = np.empty((output.GetNumberOfCells(),3,3))
for i in range(output.GetNumberOfCells()):
    #TODO: Slow, there should be a better way to convert to numpy
    cell = output.GetCell(i)
    cell_points = cell.GetPoints()
    mesh[i] = np.array([cell_points.GetPoint(i) for i in range(cell_points.GetNumberOfPoints())])

# Inverse Mercator projection

### Getting the reference coordinates

Read the GIS file

In [ ]:
import json

gis_path = "gis.json"

with open(gis_path,"r") as f:
    gis = json.load(f)

reference_coordinates = gis["referenceCoordinates"]
print("Reference coordinates: ", reference_coordinates)


In [ ]:
true_ids = np.array([building["id"] for building in gis["building"]])

In [ ]:
len(np.setdiff1d(np.unique(bids),true_ids)), len(np.setdiff1d(true_ids,np.unique(bids)))

Inverse Mercator projection

In [ ]:
import numpy as np


R_MAJOR = 6378137.0
R_MINOR=6356752.3142
ECCENT=np.sqrt(1-(R_MINOR/R_MAJOR)**2)
COM = 0.5 * ECCENT
SCALE_FACTOR = 1./np.cos(np.deg2rad(reference_coordinates[0]))


def xToLon(x):
    return np.rad2deg(x*SCALE_FACTOR/R_MAJOR)

def yToLat(y):
    return np.rad2deg(np.pi/2 - 2 * np.arctan(np.exp( -y / R_MAJOR)))

Validate

In [ ]:
latlon_mesh = np.empty((mesh.shape[0],mesh.shape[1],3))

latlon_mesh[:,:,0] = xToLon(mesh[:,:,0]) + reference_coordinates[1]
latlon_mesh[:,:,1] = yToLat(mesh[:,:,1]) + reference_coordinates[0]
latlon_mesh[:,:,2] = mesh[:,:,2]

In [ ]:
import plotly.graph_objects as go

go.Figure(
    data=[
        go.Scattermapbox(
            lon=latlon_mesh[:10000,0,0],
            lat=latlon_mesh[:10000,0,1],
            mode='markers',
            marker=dict(size=5,color='red'),
        )
    ],
    layout = go.Layout(
        margin ={'l':0,'t':0,'b':0,'r':0},
        mapbox = {
            'style': "open-street-map",
            'center': {'lon': reference_coordinates[1], 'lat': reference_coordinates[0]},
            'zoom': 13}
    )
).show()

Free the memory (Assuming we will convert the coordinates after)

## Panda Dataframes

In [ ]:
# import numpy as np
# import pandas as pd

# # Initialize an empty list to accumulate the arrays
# data = []

# for t in range(timesteps):
#     reader.SetTimeValue(time.GetValue(t))
#     reader.Update()
#     output = reader.GetOutput().GetBlock(0)
#     cell_data = output.GetCellData()
#     sc = cell_data.GetArray("solar_shading_coeff")


#     for i in range(output.GetNumberOfCells()):
#         cell = output.GetCell(i)
#         pts = cell.GetPoints()
#         scalar = sc.GetTuple(i)
#         np_pts = np.array([pts.GetPoint(j) for j in range(pts.GetNumberOfPoints())])
        
#         flattened_pts = np_pts.flatten()
#         combined_data = np.append(flattened_pts, scalar[0])
        
#         # Append the timestep and combined data to the list
#         data.append([t] + combined_data.tolist())
#         # data.append(np.insert(combined_data, 0, t))

# # Create column names for the DataFrame
# columns = ['timestep'] + ['x_1'] + ['y_1'] + ['z_1'] + ['x_2'] + ['y_2'] + ['z_2'] + ['x_3'] + ['y_3'] + ['z_3'] + ['scalar']

# # Convert the list to a DataFrame
# df = pd.DataFrame(data, columns=columns)

# # Function to calculate the area of a triangle given its vertices
# def calculate_triangle_area(p1, p2, p3):
#     # Calculate the vectors for two sides of the triangle
#     v1 = p2 - p1
#     v2 = p3 - p1
#     # Calculate the cross product of the vectors
#     cross_product = np.cross(v1, v2)
#     # Calculate the area of the triangle (0.5 * magnitude of the cross product)
#     area = 0.5 * np.linalg.norm(cross_product)
#     return area

# # Calculate the area for each triangle and add it as a new column
# areas = []
# for index, row in df.iterrows():
#     p1 = np.array([row['x_1'], row['y_1'], row['z_1']])
#     p2 = np.array([row['x_2'], row['y_2'], row['z_2']])
#     p3 = np.array([row['x_3'], row['y_3'], row['z_3']])
#     area = calculate_triangle_area(p1, p2, p3)
#     areas.append(area)

# df['area'] = areas

# # Print the resulting DataFrame
# print(df.head())


**To DataFrame**

Do this only if you REALLY need to have the data in tabular format. Otherwise, keep working with numpy.

When having a tabular format, you are using approx. 11GB of RAM, when you could easily only use approx 1.25GB.

Flatten the mesh

In [ ]:
flat_mesh = mesh.reshape(-1,mesh.shape[1]*mesh.shape[2])
print("Flattened mesh shape: ",flat_mesh.shape)

In [ ]:
import pandas as pd

data = []

for t in range(timesteps):
    reader.SetTimeValue(time.GetValue(t))
    reader.Update()
    output = reader.GetOutput().GetBlock(0)
    cell_data = output.GetCellData()
    sc_field = cell_data.GetArray("solar_shading_coeff")

    sc_array = np.empty((output.GetNumberOfCells(),1))
    for i in range(output.GetNumberOfCells()):
        sc_array[i] = sc_field.GetTuple(i)[0]

    assert sc_array.shape[0] == flat_mesh.shape[0], "Solar shading coefficient and mesh do not have the same number of cells"

    tmp_data = np.hstack(((np.ones(sc_array.shape[0])*t).reshape(-1,1),flat_mesh))
    tmp_data = np.hstack((tmp_data,sc_array))
    data.append(tmp_data)


    if t % 50 == 0:
        print("Processed timestep ", t)


del tmp_data
del sc_array

Creating the DataFrame

In [ ]:
columns=['timestep','x_1','y_1','z_1','x_2','y_2','z_2','x_3','y_3','z_3','scalar']
df = pd.DataFrame(np.vstack(data).reshape(-1,len(columns)),columns=columns)
del data

**Compute the triangle area**

In [ ]:
areas = 0.5*np.linalg.norm(
    np.cross(
        mesh[:,1]-mesh[:,0],
        mesh[:,2]-mesh[:,0]
    ), axis=1
)

df["area"] = np.array([areas]*timesteps).reshape(-1,1)

Validate

In [ ]:
df.shape

In [ ]:
df.tail(10)

To CSV

In [ ]:
df.to_csv('initial_dataframe.csv', index=False)

## Using Numpy

In [ ]:
solar_shading_in_time = np.empty((mesh.shape[0],timesteps))

for t in range(timesteps):
    reader.SetTimeValue(time.GetValue(t))
    reader.Update()
    output = reader.GetOutput().GetBlock(0)
    cell_data = output.GetCellData()
    sc_field = cell_data.GetArray("solar_shading_coeff")

    for i in range(output.GetNumberOfCells()):
        solar_shading_in_time[i,t] = sc_field.GetTuple(i)[0]

## Some example statistics

In [ ]:
sample_size = 10000
latlon_mesh_sample = latlon_mesh[:sample_size]
solar_shading_sample = solar_shading_in_time[:sample_size]
t = 130

#This takes all faces, so points in a 2d plot will be repeated.

In [ ]:

go.Figure(
    data=[
        go.Scattermapbox(
            lon=latlon_mesh_sample[:,0,0],
            lat=latlon_mesh_sample[:,0,1],
            mode='markers',
            marker=dict(size=6,color=solar_shading_sample[:,t],colorscale='rdbu',cmin=0,cmax=1),
        )
    ],
    layout = go.Layout(
        margin ={'l':0,'t':0,'b':0,'r':0},
        mapbox = {
            'style': "open-street-map",
            'center': {'lon': reference_coordinates[1], 'lat': reference_coordinates[0]},
            'zoom': 13},
    )
).show()